In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [3]:
df_estado = spark.read.format('parquet').load('s3a://silver/dado_geografico/estado')
# df_estado.printSchema()
df_estado.show(5, False)

+---------+-----------+------------+---------+-----------+------------+
|estado_id|estado_nome|estado_sigla|regiao_id|regiao_nome|regiao_sigla|
+---------+-----------+------------+---------+-----------+------------+
|11       |Rondônia   |RO          |1        |Norte      |N           |
|12       |Acre       |AC          |1        |Norte      |N           |
|13       |Amazonas   |AM          |1        |Norte      |N           |
|14       |Roraima    |RR          |1        |Norte      |N           |
|15       |Pará       |PA          |1        |Norte      |N           |
+---------+-----------+------------+---------+-----------+------------+
only showing top 5 rows



In [4]:
df_municipio = spark.read.format('parquet').load('s3a://silver/dado_geografico/municipio')
# df_municipio.printSchema()
# print(df_municipio.dtypes)
df_municipio.show(5, False)

+------------+---------------------+--------------+-----------------+-------------------------+---------------------------+----------------------------+-------------------------------+------------------------------+-----------------------------------+--------------------------------------+-------------------------------------+------------------+--------------------+--------------------------------+----------------------------------+-----------------------------------+--------------------------------------+-------------------------------------+------------------------------------------+---------------------------------------------+--------------------------------------------+
|municipio_id|municipio_nome       |microregiao_id|microregiao_nome |microregiao_mesoregiao_id|microregiao_mesoregiao_nome|microregiao_mesoregiao_uf_id|microregiao_mesoregiao_uf_sigla|microregiao_mesoregiao_uf_nome|microregiao_mesoregiao_uf_regiao_id|microregiao_mesoregiao_uf_regiao_sigla|microregiao_mesoregiao_uf_

In [6]:
df_fluxo_educ_superior = spark.read.format('parquet').load('s3a://bronze/inep/educacao_superior/indicadores_fluxo_educacao_superior')
# df_fluxo_educ_superior.printSchema()
# print(df_fluxo_educ_superior.dtypes)
df_fluxo_educ_superior.show(5, False)

+------+-----------------------------------+---------------------------+------------------------+--------+--------+---------+-----+------------+-----------------+--------------------+--------------+--------------+------------------+---------------------------------+---------------+-----------------+-----------------------+---------------------+-----------------------+----------------------------+--------------+--------------+-------------+--------------+-----------+------------------+------------------+------------------+------------------+------------------+
|CO_IES|NO_IES                             |TP_CATEGORIA_ADMINISTRATIVA|TP_ORGANIZACAO_ACADEMICA|CO_CURSO|NO_CURSO|CO_REGIAO|CO_UF|CO_MUNICIPIO|TP_GRAU_ACADEMICO|TP_MODALIDADE_ENSINO|CO_CINE_ROTULO|NO_CINE_ROTULO|CO_CINE_AREA_GERAL|NO_CINE_AREA_GERAL               |NU_ANO_INGRESSO|NU_ANO_REFERENCIA|NU_PRAZO_INTEGRALIZACAO|NU_ANO_INTEGRALIZACAO|NU_PRAZO_ACOMPANHAMENTO|NU_ANO_MAXIMO_ACOMPANHAMENTO|QT_INGRESSANTE|QT_PERMANENCIA|QT_CO

In [7]:
# df_fluxo_educ_superior.dtypes

In [8]:
# df_fluxo_educ_superior = (df_fluxo_educ_superior
#     .select(fn.col('CO_IES').cast('bigint').alias('CO_IES'),
#             fn.col('NO_IES').cast('string').alias('NO_IES'),
#             fn.col('TP_CATEGORIA_ADMINISTRATIVA').cast('int').alias('TP_CATEGORIA_ADMINISTRATIVA'),
#             fn.col('TP_ORGANIZACAO_ACADEMICA').cast('int').alias('TP_ORGANIZACAO_ACADEMICA'),
#             fn.col('CO_CURSO').cast('bigint').alias('CO_CURSO'),
#             fn.col('NO_CURSO').cast('string').alias('NO_CURSO'),
#             fn.col('CO_REGIAO').cast('int').alias('CO_REGIAO'),
#             fn.col('CO_UF').cast('int').alias('CO_UF'),
#             fn.col('CO_MUNICIPIO').cast('bigint').alias('CO_MUNICIPIO'),
#             fn.col('TP_GRAU_ACADEMICO').cast('int').alias('TP_GRAU_ACADEMICO'),
#             fn.col('TP_MODALIDADE_ENSINO').cast('int').alias('TP_MODALIDADE_ENSINO'),
#             fn.col('CO_CINE_ROTULO').cast('string').alias('CO_CINE_ROTULO'),
#             fn.col('NO_CINE_ROTULO').cast('string').alias('NO_CINE_ROTULO'),
#             fn.col('CO_CINE_AREA_GERAL').cast('int').alias('CO_CINE_AREA_GERAL'),
#             fn.col('NO_CINE_AREA_GERAL').cast('string').alias('NO_CINE_AREA_GERAL'),
#             fn.col('NU_ANO_INGRESSO').cast('int').alias('NU_ANO_INGRESSO'),
#             fn.col('NU_ANO_REFERENCIA').cast('int').alias('NU_ANO_REFERENCIA'),
#             fn.col('NU_PRAZO_INTEGRALIZACAO').cast('int').alias('NU_PRAZO_INTEGRALIZACAO'),
#             fn.col('NU_ANO_INTEGRALIZACAO').cast('int').alias('NU_ANO_INTEGRALIZACAO'),
#             fn.col('NU_PRAZO_ACOMPANHAMENTO').cast('int').alias('NU_PRAZO_ACOMPANHAMENTO'),
#             fn.col('NU_ANO_MAXIMO_ACOMPANHAMENTO').cast('int').alias('NU_ANO_MAXIMO_ACOMPANHAMENTO'),
#             fn.col('QT_INGRESSANTE').cast('bigint').alias('QT_INGRESSANTE'),
#             fn.col('QT_PERMANENCIA').cast('bigint').alias('QT_PERMANENCIA'),
#             fn.col('QT_CONCLUINTE').cast('bigint').alias('QT_CONCLUINTE'),
#             fn.col('QT_DESISTENCIA').cast('bigint').alias('QT_DESISTENCIA'),
#             fn.col('QT_FALECIDO').cast('bigint').alias('QT_FALECIDO'),
#             fn.col('TAP').cast('double').alias('TAP'),
#             fn.col('TCA').cast('double').alias('TCA'),
#             fn.col('TDA').cast('double').alias('TDA'),
#             fn.col('TCAN').cast('double').alias('TCAN'),
#             fn.col('TADA').cast('double').alias('TADA')
#            )
# )

In [9]:
print('df_fluxo_educ_superior.count(): ', df_fluxo_educ_superior.count())

df_fluxo_educ_superior.count():  259230


In [10]:
df_fluxo_educ_superior_estmun = (df_fluxo_educ_superior.alias('educ')
     .join(df_estado.alias('est'), fn.col('educ.CO_UF') == fn.col('est.estado_id'), 'left')
     .join(df_municipio.alias('mun'), fn.col('educ.CO_MUNICIPIO') == fn.col('mun.municipio_id'), 'left')
     .select(fn.col('educ.CO_IES').alias('codigo_instituicao'),
             fn.col('educ.NO_IES').alias('nome_instituicao'),
             fn.col('educ.TP_CATEGORIA_ADMINISTRATIVA').alias('categoria_administrativa'),
             fn.col('educ.TP_ORGANIZACAO_ACADEMICA').alias('organizacao_academica'),
             fn.col('educ.CO_CURSO').alias('codigo_curso_graduacao'),
             fn.col('educ.NO_CURSO').alias('nome_curso_graduacao'),
             fn.col('educ.CO_REGIAO').alias('codigo_regiao_geografica_curso'),
             fn.col('est.regiao_nome').alias('nome_regiao_geografica_curso'), # df_estado.regiao_nome
             fn.col('educ.CO_UF').alias('codigo_unidade_federativa_curso'),
             fn.col('est.estado_nome').alias('nome_unidade_federativa_curso'), # df_estado.estado_nome
             fn.col('educ.CO_MUNICIPIO').alias('codigo_municipio_curso'),
             fn.col('mun.municipio_nome').alias('nome_municipio_curso'), # df_municipio.municipio_nome
             fn.col('educ.TP_GRAU_ACADEMICO').alias('tipo_grau_academico'),
             fn.col('educ.TP_MODALIDADE_ENSINO').alias('tipo_modalidade_ensino'),
             fn.col('educ.CO_CINE_ROTULO').alias('codigo_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NO_CINE_ROTULO').alias('nome_area_curso_classificacao_cine_brasil'),
             fn.col('educ.CO_CINE_AREA_GERAL').alias('codigo_grande_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NO_CINE_AREA_GERAL').alias('nome_grande_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NU_ANO_INGRESSO').alias('ano_ingresso'),
             fn.col('educ.NU_ANO_REFERENCIA').alias('ano_referencia'),
             fn.col('educ.NU_PRAZO_INTEGRALIZACAO').alias('prazo_integralizacao_anos'),
             fn.col('educ.NU_ANO_INTEGRALIZACAO').alias('ano_integralizacao_curso'),
             fn.col('educ.NU_PRAZO_ACOMPANHAMENTO').alias('prazo_acompanhamento_curso_anos'),
             fn.col('educ.NU_ANO_MAXIMO_ACOMPANHAMENTO').alias('ano_maximo_acompanhamento_curso'),
             fn.col('educ.QT_INGRESSANTE').alias('quantidade_ingressante_curso'),
             fn.col('educ.QT_PERMANENCIA').alias('quantidade_permanência_curso_ano_referencia'),
             fn.col('educ.QT_CONCLUINTE').alias('quantidade_concluinte_curso_ano_referencia'),
             fn.col('educ.QT_DESISTENCIA').alias('quantidade_desistencia_curso_ano_referencia'),
             fn.col('educ.QT_FALECIDO').alias('quantidade_falecimento_curso_ano_referencia'),
             fn.col('educ.TAP').alias('taxa_permanencia_tap'),
             fn.col('educ.TCA').alias('taxa_conclusao_acumulada_tca'), 
             fn.col('educ.TDA').alias('taxa_desistencia_acumulada_tda'),
             fn.col('educ.TCAN').alias('taxa_conclusso_anual_tcan'),
             fn.col('educ.TADA').alias('taxa_desistencia_anual_tada')
            )
)
print('df_fluxo_educ_superior_estmun.count(): ', df_fluxo_educ_superior_estmun.count())
# df_fluxo_educ_superior_estmun.printSchema()
# print(df_fluxo_educ_superior_estmun.dtypes)
df_fluxo_educ_superior_estmun.show(12, False)

df_fluxo_educ_superior_estmun.count():  259230
+------------------+-----------------------------------+------------------------+---------------------+----------------------+--------------------+------------------------------+----------------------------+-------------------------------+-----------------------------+----------------------+--------------------+-------------------+----------------------+-------------------------------------------+-----------------------------------------+--------------------------------------------------+------------------------------------------------+------------+--------------+-------------------------+------------------------+-------------------------------+-------------------------------+----------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------+-------------------------------------------+--------------------+----------------------------+--------------